In [1]:
import numpy as np 
import cv2 
import os
import re

In [2]:
dataset_dir = "dataset_cropping" 
name = "bayam_merah30.jpg"

In [3]:
img = cv2.imread(os.path.join(dataset_dir,"Bayam Merah",name))

In [4]:
gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

In [5]:
h, w = gray.shape

In [6]:
ymin, ymax, xmin, xmax = h//150, h*149//150, w//150, w*149//150    

In [7]:
crop = gray[ymin:ymax, xmin:xmax]
resize = cv2.resize(crop, (0,0), fx=0.2, fy=0.2)

In [8]:
imgs = [] #list image matrix 
labels = []
descs = []
imgs.append(resize)

In [9]:
# cv2.imshow(name, resize)
# cv2.waitKey(0)
# cv2.destroyAllWindows() 

In [10]:
def normalize_label(str_):
    str_ = str_.replace(" ", "")
    str_ = str_.translate(str_.maketrans("","", "()"))
    str_ = str_.split("_")
    return re.sub(r'\d+$', '',''.join(str_[:2]))

labels.append(normalize_label(os.path.splitext(name)[0]))

In [11]:
from skimage.feature import greycomatrix, greycoprops

# ----------------- calculate greycomatrix() & greycoprops() for angle 0, 45, 90, 135 ----------------------------------
def calc_glcm_all_agls(img, label, props, dists=[5], agls=[0, np.pi/4, np.pi/2, 3*np.pi/4], lvl=256, sym=True, norm=True):
    glcm = greycomatrix(img, 
                        distances=dists, 
                        angles=agls, 
                        levels=lvl,
                        symmetric=sym, 
                        normed=norm)
    feature = []
    glcm_props = [propery for name in props for propery in greycoprops(glcm, name)[0]]
    for item in glcm_props:
            feature.append(item)
    feature.append(label) 
    
    return feature


# ----------------- call calc_glcm_all_agls() for all properties ----------------------------------
properties = ['dissimilarity', 'correlation', 'homogeneity', 'contrast', 'ASM', 'energy']

glcm_all_agls = []
for img, label in zip(imgs, labels): 
    glcm_all_agls.append(
            calc_glcm_all_agls(img, 
                                label, 
                                props=properties)
                            )
 
columns = []
angles = ['0', '45', '90','135']
for name in properties :
    for ang in angles:
        columns.append(name + "_" + ang)
        
columns.append("label")


In [12]:
import pandas as pd 

# Create the pandas DataFrame for GLCM features data
glcm_df = pd.DataFrame(glcm_all_agls, 
                      columns = columns)

#save to csv
# glcm_df.to_csv("glcm_daun_herbal2.csv")

In [13]:
glcm_df.head()

,dissimilarity_0,dissimilarity_45,dissimilarity_90,dissimilarity_135,correlation_0,correlation_45,correlation_90,correlation_135,homogeneity_0,homogeneity_45,...,contrast_135,ASM_0,ASM_45,ASM_90,ASM_135,energy_0,energy_45,energy_90,energy_135,label
0,1.823612,1.799974,1.599251,1.867987,0.968657,0.969226,0.9731,0.967216,0.910344,0.91043,...,203.114124,0.799544,0.798817,0.80096,0.798584,0.894172,0.893766,0.894964,0.893635,bayammerah


In [15]:
X = glcm_df.iloc[:, :-1].values  
y = glcm_df.iloc[:, 24].values

print(X,"\n\n", y)

[[  1.82361158   1.7999739    1.5992509    1.86798669   0.96865739
    0.96922578   0.97309975   0.9672157    0.91034443   0.91042983
    0.9121867    0.91004636 193.46924296 190.66078419 165.66284016
  203.11412448   0.79954356   0.79881702   0.80095968   0.79858353
    0.894172     0.89376564   0.89496351   0.89363501]] 

 ['bayammerah']
